# Imports and init

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import xarray as xr

from matplotlib import pyplot as plt

import holoviews as hv
import hvplot.xarray
from param import Parameter, Parameterized
import panel as pn

In [3]:
from labstack.data.datadict import str2dd, DataDict, datadict_to_meshgrid, MeshgridDataDict
from labstack.testing.dispersive_qubit_readout_data import angle_data, probability_data

# Make and look at fake Rabi data

In [4]:
dd = str2dd("signal(repetition, rabi_angle); rabi_angle[rad]; ")

nreps = 100
for theta in np.linspace(-2*np.pi, 2*np.pi, 41):
    dd.add_data(
        rabi_angle=[theta], 
        repetition=np.arange(nreps).reshape(1,-1).astype(int),
        signal=angle_data(theta, nreps),
    )
dd = datadict_to_meshgrid(dd)

## manual plot to sanity-check

In [ ]:
dd_avg = dd.mean('repetition')

fig, axes = plt.subplot_mosaic(
    [
        ['t_evo', 'hist']
    ],
    constrained_layout=True,
    figsize=(6,3),
)

axes['t_evo'].plot(
    dd_avg.d_.rabi_angle, dd_avg.d_.signal.real, '.-',
    label='Re',
)
axes['t_evo'].plot(
    dd_avg.d_.rabi_angle, dd_avg.d_.signal.imag, '.-',
    label='Im',
)
axes['t_evo'].legend(loc='best')

h, xe, ye, im = axes['hist'].hist2d(dd.d_.signal.real.flatten(), dd.d_.signal.imag.flatten(), bins=51)
axes['hist'].set_aspect('equal')
cb = fig.colorbar(im, shrink=0.5)

## convert the data to xarray

In [5]:
ds = xr.Dataset(
    {
        "signal": (["repetition", "rabi_angle"], dd.d_.signal)
    },
    coords={
        "repetition": dd.d_.repetition[:,0],
        "rabi_angle": dd.d_.rabi_angle[0,:],
    }
)

ds

<xarray.Dataset>
Dimensions:     (repetition: 100, rabi_angle: 41)
Coordinates:
  * repetition  (repetition) int64 0 1 2 3 4 5 6 7 8 ... 92 93 94 95 96 97 98 99
  * rabi_angle  (rabi_angle) float64 -6.283 -5.969 -5.655 ... 5.655 5.969 6.283
Data variables:
    signal      (repetition, rabi_angle) complex128 (1.922820062411938+0.5923...

## use hvPlot to get a quick look at the raw data

In [6]:
(ds.real.hvplot.quadmesh(width=300) + ds.imag.hvplot.quadmesh(width=300)).cols(2)

:Layout
   .QuadMesh.I  :QuadMesh   [rabi_angle,repetition]   (signal)
   .QuadMesh.II :QuadMesh   [rabi_angle,repetition]   (signal)

In [7]:
ds_ = ds.mean('repetition')

((ds_.real.hvplot.line(width=500) * ds_.real.hvplot.scatter())
  * (ds_.imag.hvplot.line() * ds_.imag.hvplot.scatter()))

:Overlay
   .Curve.I    :Curve   [rabi_angle]   (signal)
   .Scatter.I  :Scatter   [rabi_angle]   (signal)
   .Curve.II   :Curve   [rabi_angle]   (signal)
   .Scatter.II :Scatter   [rabi_angle]   (signal)

## alternative: split Re/Im

In [8]:
ds2 = xr.Dataset(
    {
        "Real": (["repetition", "rabi_angle"], dd.d_.signal.real),
        "Imag": (["repetition", "rabi_angle"], dd.d_.signal.imag),
    },
    coords={
        "repetition": dd.d_.repetition[:,0],
        "rabi_angle": dd.d_.rabi_angle[0,:],
    }
)

ds2

<xarray.Dataset>
Dimensions:     (repetition: 100, rabi_angle: 41)
Coordinates:
  * repetition  (repetition) int64 0 1 2 3 4 5 6 7 8 ... 92 93 94 95 96 97 98 99
  * rabi_angle  (rabi_angle) float64 -6.283 -5.969 -5.655 ... 5.655 5.969 6.283
Data variables:
    Real        (repetition, rabi_angle) float64 1.923 2.612 ... 1.367 2.917
    Imag        (repetition, rabi_angle) float64 0.5923 0.4382 ... -0.7142

In [9]:
ds2.hvplot(z='Real', clabel='Re', width=300) + ds2.hvplot(z='Imag', clabel='Im', width=300)

:Layout
   .Image.I  :Image   [rabi_angle,repetition]   (Real)
   .Image.II :Image   [rabi_angle,repetition]   (Imag)

In [10]:
plot = ds2.mean('repetition').hvplot.line(grid=True, width=500) * ds2.mean('repetition').hvplot.scatter()
plot

:Overlay
   .NdOverlay.I  :NdOverlay   [Variable]
      :Curve   [rabi_angle]   (value)
   .NdOverlay.II :NdOverlay   [Variable]
      :Scatter   [rabi_angle]   (value)

## Example app: average selector

In [11]:
def plot_all(dataset):
    if len(dataset.dims) == 1:
        return dataset.hvplot.line(width=500) * dataset.hvplot.scatter()
    else:
        dvars = list(dataset.data_vars.keys())
        ret = dataset[dvars[0]].hvplot(width=500)
        for d in dvars[1:]:
            ret += dataset[d].hvplot(width=500)
        return ret.cols(1)
        
def avg_and_plot(dataset, dim=None):
    if dim in [None, 'None']:
        return plot_all(dataset)
    else:
        return plot_all(dataset.mean(dim))


available_dims = list(ds2.dims.keys())
avg_selector = pn.widgets.Select(name='Averaging dimension', value='None', options=['None'] + available_dims)
plot = pn.bind(avg_and_plot, ds2, dim=avg_selector)

app = pn.Column(avg_selector, plot)

app

Column
    [0] Select(name='Averaging dimension', options=['None', 'repetition', ...], value='None')
    [1] ParamFunction(function, _pane=HoloViews)

# Inspecting a more-dimensional dataset

## make fake Chevron data

In [12]:
twopi = np.pi * 2

def rabi(Omega_0, Delta, t):
    Omega = (Omega_0**2 + Delta**2)**.5 
    return (Omega_0 / Omega)**2 * (1.0+np.cos(Omega*t))/2.

def make_chevron_data(Omega_0, Delta_vals, t_vals, n):
    shp = (n, Delta_vals.size, t_vals.size)
    print(shp)
    data = np.empty(shp, dtype=complex)
    for i, Delta in enumerate(Delta_vals):
        for j, t in enumerate(t_vals):
            data[:, i, j] = probability_data(rabi(Omega_0, Delta, t), n=n)
    return data

def make_chevron_ds(Omega_0, Delta_vals, t_vals, n):
    arr = make_chevron_data(twopi*Omega_0, twopi*Delta_vals, t_vals, n)
    ds = xr.Dataset(
        {
            "readout_real": (["repetition", "detuning", "time"], arr.real,),
            "readout_imag": (["repetition", "detuning", "time"], arr.imag,),
        },
        coords={
            "repetition": np.arange(n, dtype=int)+1,
            "detuning": Delta_vals,
            "time": t_vals,
        }
    )
    return ds
    
ds = make_chevron_ds(
    1,
    np.linspace(-3, 3, 51),
    np.linspace(0, 3, 31),
    500,
)

ds

(500, 51, 31)


<xarray.Dataset>
Dimensions:       (repetition: 500, detuning: 51, time: 31)
Coordinates:
  * repetition    (repetition) int64 1 2 3 4 5 6 7 ... 495 496 497 498 499 500
  * detuning      (detuning) float64 -3.0 -2.88 -2.76 -2.64 ... 2.76 2.88 3.0
  * time          (time) float64 0.0 0.1 0.2 0.3 0.4 0.5 ... 2.6 2.7 2.8 2.9 3.0
Data variables:
    readout_real  (repetition, detuning, time) float64 2.273 2.501 ... 2.055
    readout_imag  (repetition, detuning, time) float64 0.364 1.256 ... 0.2456

## inspect without custom code

In [ ]:
dsh_ =  ds.mean('repetition').hvplot
plot = dsh_.image(x='detuning', z='readout_real')
plot

In [ ]:
dsh_.line(x='time') * dsh_.scatter(x='time')

## inspect with own app

In [16]:
from labstack.analysis.plotting.holo import SimplePlot

sp = SimplePlot()
sp.data = ds
sp.make()

Column
    [0] GridBox(ncols=2)
        [0] Markdown(str)
        [1] RadioButtonGroup(name='repetition', options=['None', 'Mean'], value='None')
        [2] Markdown(str)
        [3] RadioButtonGroup(name='detuning', options=['None', 'Mean'], value='None')
        [4] Markdown(str)
        [5] RadioButtonGroup(name='time', options=['None', 'Mean'], value='None')
    [1] ParamFunction(function, _pane=Row)

# Scribble

In [27]:
import param
import panel as pn
from panel.viewable import Viewer
from panel.widgets import RadioButtonGroup as RBG

class XYSelect(Viewer):
    value = param.Tuple(default=(None, None))
    options = param.List(default=[None,])
    
    def __init__(self, **params):
        params['options'] = ['None'] + params.get('options', [])
        self.opts = params
        self._xrbg = RBG(options=params['options'])
        self._yrbg = RBG(options=params['options'])
        super().__init__(**params)
        self._layout = pn.Column(
            '### X dimension',
            self._xrbg,
            '### Y dimension',
            self._yrbg,
        )
        
    def __panel__(self):
        return self._layout
    
    @param.depends('value', 'options', watch=True)
    def _sync_widgets(self):
        if self.value[0] == self.value[1]:
            raise ValueError('X and Y dims cannot be the same.')
        self._xrbg.name  = self.name
        self._xrbg.value = self.value[0]
        self._yrbg.value = self.value[1]
        
    @param.depends('_xrbg.value', watch=True)
    def _sync_x(self):
        x = self._xrbg.value
        if self.value[1] == x:
            pass
        self.value = (self._xrbg.value, None)
        

xy = XYSelect(options=['reps', 'time', 'det'])
xy

XYSelect(name='XYSelect08881', options=['None', 'reps', 'time', 'det'], value=(None, None))